In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch.nn as nn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
train_data = pd.read_csv('train_df.csv')
test_data = pd.read_csv('test_df.csv')

In [5]:
train_data = train_data.drop(columns=['fecha_dato'])
test_data = test_data.drop(columns=['fecha_dato'])

In [6]:
# Scale dữ liệu và label các giá trị string

# Thêm cột đánh dấu nguồn dữ liệu
train_data["source"] = "train"
test_data["source"] = "test"

# Ghép nối tiếp hai DataFrame theo hàng
df = pd.concat([train_data, test_data], ignore_index=True)
# Loại bỏ các cột không cần thiết trước khi xử lý
df_process = df.drop(columns=['ncodpers', 'source', 'output'], errors='ignore')

# Label Encode các cột chuỗi
str_columns = df_process.select_dtypes(exclude=['number']).columns.tolist()
encoder = LabelEncoder()
for col in str_columns:
    df_process[col] = encoder.fit_transform(df_process[col])

# Scale dữ liệu
scaler = MinMaxScaler()
scaled_ndarray = scaler.fit_transform(df_process)
df_process = pd.DataFrame(scaled_ndarray, columns=df_process.columns)

# Thêm lại các cột 'ncodpers', 'output' và 'source'
df_process['ncodpers'] = df['ncodpers']
df_process['source'] = df['source']
df_process['output'] = df['output']
train_data_process = df_process[df_process['source'] == 'train'].drop(columns=['source'])
test_data_process = df_process[df_process['source'] == 'test'].drop(columns=['source'])

In [7]:
print(train_data_process.head().to_string())

      renta  sexo       age  antiguedad  indrel  indresi  indext  indfall  cod_prov  segmento  ind_ahor_fin_ult1  ind_aval_fin_ult1  ind_cco_fin_ult1  ind_cder_fin_ult1  ind_cno_fin_ult1  ind_ctju_fin_ult1  ind_ctma_fin_ult1  ind_ctop_fin_ult1  ind_ctpp_fin_ult1  ind_deco_fin_ult1  ind_deme_fin_ult1  ind_dela_fin_ult1  ind_ecue_fin_ult1  ind_fond_fin_ult1  ind_hip_fin_ult1  ind_plan_fin_ult1  ind_pres_fin_ult1  ind_reca_fin_ult1  ind_tjcr_fin_ult1  ind_valo_fin_ult1  ind_viv_fin_ult1  ind_nomina_ult1  ind_nom_pens_ult1  ind_recibo_ult1  inti_1mes  ind_empleado_B  ind_empleado_F  ind_empleado_N  ind_empleado_S  ncodpers             output
0  0.004400   1.0  0.442478    0.964706     1.0      1.0     0.0      0.0  0.666667       0.5                0.0                0.0               1.0                0.0               0.0                0.0                0.0                0.0                0.0                0.0                0.0                0.0                0.0                

In [8]:
test_acc = pd.read_csv('test_acc.csv')

In [10]:
customer_feature = ['renta', 'ind_empleado_B', 'ind_empleado_F', 'ind_empleado_N', 'ind_empleado_S', 'sexo', 'age',
       'antiguedad', 'indrel', 'inti_1mes', 'indresi', 'indext', 'indfall', 'cod_prov', 'segmento', 
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 
       'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

X_train = train_data_process.loc[:,customer_feature]   # Đặc trưng khách hàng
Y_train = train_data_process['output']  # Nhãn sản phẩm
X_test = test_data_process.loc[:,customer_feature]   # Đặc trưng khách hàng
Y_test = test_data_process['output']    # Nhãn sản phẩm

# Label cột output
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

# Khởi tạo mô hình
xgb = XGBClassifier(
    objective="multi:softmax",  # Sử dụng softmax cho bài toán phân loại đa lớp
    num_class=len(encoder.classes_),  # Số lượng lớp đầu ra
    eval_metric="mlogloss",  # Metric đánh giá
    learning_rate=0.05,  # Tốc độ học, có thể giảm xuống 0.05 nếu cần
    max_depth=3,  # Độ sâu tối đa của cây, tăng lên để phức tạp hóa mô hình
    n_estimators=120,  # Số lượng cây, tăng lên để cải thiện hiệu suất
) # , learning_rate=0.05, max_depth=3, n_estimators=120 - tham số này học được từ grid_search đã chạy

rf = RandomForestClassifier(
    n_estimators=70,  # Số lượng cây, tăng lên để cải thiện hiệu suất
    max_depth=10,  # Độ sâu tối đa của cây, tăng lên để phức tạp hóa mô hình
) # n_estimators=70, max_depth=10 - tham số này học được từ grid_search đã chạy

# Tạo Voting Classifier với XGBoost, RandomForest và LightGBM
ensemble_model = VotingClassifier(estimators=[
    ('xgb', xgb),
    ('rf', rf)
], voting='soft')  # Soft voting sử dụng xác suất dự đoán

# # GridSearchCV - Tìm bộ tham số tối ưu
# param_grid = {
#     'xgb__n_estimators': [100, 200],  
#     'xgb__max_depth': [3, 6],  
#     'xgb__learning_rate': [0.05, 0.1],  
#     'rf__n_estimators': [100, 200],  
#     'rf__max_depth': [10, 20]  
# }
# grid_search = GridSearchCV(ensemble_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
# grid_search.fit(X_train, Y_train)

# # In kết quả tốt nhất
# print("Best parameters:", grid_search.best_params_)
# print("Best accuracy:", grid_search.best_score_)
# best_model = grid_search.best_estimator_ # Kết quả: Best parameters: {'rf__max_depth': 10, 'rf__n_estimators': 70, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 3, 'xgb__n_estimators': 120}

# Huấn luyện mô hình
ensemble_model.fit(X_train, Y_train)


VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='mlogloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=120, n_jobs=None,
                                            num_class=24,
                                            num_parallel_tree=None, ...)),
                             ('rf',
                              RandomForestClassifier(max_depth=10,
                                                     n_estimators=70))],
                 voting='soft')

In [11]:
# Dự đoán xác suất mua hàng
Y_pred_proba = ensemble_model.predict(X_test)  # Danh sách 20 mảng xác suất
Y_pred_proba = encoder.inverse_transform(Y_pred_proba) 

# Chuyển kết quả thành DataFrame
Y_pred_proba_df = pd.DataFrame(Y_pred_proba, columns=['output'])
predict_df = Y_pred_proba_df
predict_df['ncodpers'] = test_data['ncodpers']

In [12]:
# Đánh giá mô hình
predict_df = predict_df.drop_duplicates()
# Đếm số dòng có giá trị cột 'output' giống nhau giữa df1 và df2
num_correct = 0
for index, row in predict_df.iterrows():
    ncodpers = row['ncodpers']
    output = row['output']
    match = test_acc[(test_acc['ncodpers']==ncodpers) & (test_acc['output']==output)]
    if not match.empty:
        num_correct += 1

# Tổng số dòng
total_rows = len(predict_df)

# Tỉ lệ đoán đúng
accuracy = num_correct / total_rows

print(f"Số dòng giống nhau: {num_correct}")
print(f"Tổng số dòng: {total_rows}")
print(f"Tỉ lệ đoán đúng: {accuracy:.2%}")  # Hiển thị dạng %



Số dòng giống nhau: 20661
Tổng số dòng: 27875
Tỉ lệ đoán đúng: 74.12%


In [ ]:
predict_df.to_csv('predict.csv')